In [1]:
import cv2
import os 
import numpy as np
import glob


In [2]:
subjects = [" ","Samridhi Prasad","Raksha Maheshwari","Sakshi Gupta"]

In [3]:
#function to dectect faces
def face_detect(image):
    cascPath = "haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(cascPath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=5,
        minSize=(30, 30)
    );
    if len(faces)== None:
        return None,None

    (x, y, w, h)=faces[0]
    return gray[y:y+w,x:x+h],faces
    
    


In [4]:
#function to prepare training data
def prepare_training_data(data_folder_path):
    #get the directories
    dirs=os.listdir(data_folder_path)
    #list to hold all faces
    faces=[]
    #list to hold all labels
    labels=[]
    print(dirs)
    #lets check each directory
    for dir_name in dirs:
        #since our directory name start with s ignore any irrelevant directory
        if not dir_name.startswith("s"):
            continue
        #extract the label number from dir_name
        label = int(dir_name.replace("s","  "))
        #build the path for the images of current directory
        subject_dir_path = data_folder_path+"/"+dir_name
        print(subject_dir_path)
        #get the image name inside the directory path
        subject_image_name=os.listdir(subject_dir_path)
        #go through each image name read image
        for image_name in subject_image_name:
            #ignore .ds file
            if image_name.startswith("."):
                continue
            #build image path
            image_path = subject_dir_path+"/"+image_name
            #read image
            image = cv2.imread(image_path)
            #display the image
            cv2.imshow("Training image ...",image)
            cv2.waitKey(100)
            #detect face
            face,rect=face_detect(image)
            if face is not None:
                #add face to the list of face
                faces.append(face)
                #add label to list of label
                labels.append(label)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    return faces,labels
    

In [5]:
#let's first prepare our training data
#data will be in two lists of same size
#one list will contain all the faces
#and the other list will contain respective labels for each face
print("Preparing data...")
faces, labels = prepare_training_data("C:/Users/Samridhi Prasad/Documents/training_data")
print("Data prepared")

#print total faces and labels
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))


Preparing data...
['s1', 's2', 's3']
C:/Users/Samridhi Prasad/Documents/training_data/s1
C:/Users/Samridhi Prasad/Documents/training_data/s2
C:/Users/Samridhi Prasad/Documents/training_data/s3
Data prepared
Total faces:  32
Total labels:  32


In [6]:
#train face recogniser
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
#train our face recognizer of our training faces
face_recognizer.train(faces, np.array(labels))



In [7]:
#this function recognizes the person in image passed
#and draws a rectangle around detected face with name of the 
#subject
def predict(test_img):
#make a copy of the image as we don't want to change original image
    img = test_img.copy()
#detect face from the image
    face,rect = face_detect(img)
    if face is not None:
        #predict the image using our face recognizer 
        label = face_recognizer.predict(face)
        print(label)
        if int(label[1])>=70:
            #get name of respective label returned by face recognizer 
            label_text = subjects[label[0]]
        else:
            label_text=input("Please Enter Your Name:")
        return label_text
    else:
        return "No face Detected"


In [8]:
print("Predicting images...")
attendance =[]
img_names= glob.glob('C:\\Users\\Samridhi Prasad\\Documents\\project\\saveimage\\*.jpg')
print(type(img_names))
for i in img_names:
    print(i)
    temp=cv2.imread(i)
    cv2.imshow('Image',temp)
    cv2.waitKey(100)
    predict_img=predict(temp)
    if predict_img not in attendance:
        attendance.append(predict_img)
print(attendance)
print("Prediction complete")
cv2.destroyAllWindows()



Predicting images...
<class 'list'>
C:\Users\Samridhi Prasad\Documents\project\saveimage\file_0.jpg


IndexError: tuple index out of range